# **Implémentation d'un modèle de scoring**
## **Notebook 4/6 - Modélisation : Sélection des features**
*Sofia Chevrolat (Novembre 2020)*
___
Cette étude vise à réaliser un modèle permettant de prédire le risque de défaut de remboursement d'un prêt pour un client donné. Ce modèle doit être basé sur une variété de données (personnelles, en provenance de différentes sources bancaires, etc.).

Ce modèle est destiné à être servi via une API, elle-même appelée via un dashboard interactif. Le modèle devra donc être exporté une fois finalisé.
___
_**Remerciements**:<br>
Merci à mon compagnon [J. Duplan](https://www.linkedin.com/in/julian-duplan-64844a41/) pour les discussions intéressantes.<br>
Merci également à mon mentor [Samia Drappeau](https://www.linkedin.com/in/samiadrappeau) pour les échanges d'idées, les conseils et les encouragements !_
___


Ce notebook est organisé comme suit:

**0. Mise en place**
- 0.1 Chargement des librairies et fonctions utiles
- 0.2 Chargement et description du jeu de données
- 0.3 Suppression des features non pertinentes
- 0.4 Séparation des données
    
**1. Sélection des features**
- 1.1 Baseline : aucune sélection
- 1.2 Suppression des features colinéaires
- 1.3 Suppression des features présentant plus de 75% de valeurs manquantes
- 1.4 Suppression des features ayant une importance nulle pour le modèle
- 1.5 Suppression des features ayant une importance inférieure à 95% pour le modèle
- 1.6 Comparaison des performances

**2. Conclusion**

**3. Export des données**

___
### 0. MISE EN PLACE

Dans cette première étape, le cadre de travail est mis en place, c'est-à-dire :
- Les librairies et packages Python nécessaires sont chargés
- Les fonctions utiles sont définies
- Le jeu de données est chargé
___

___
#### 0.1 CHARGEMENT DES LIBRAIRIES ET FONCTIONS UTILES

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import time
import random
import gc

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
import lightgbm as lgb
from joblib import load, dump

In [4]:
from sys import path
path.append("./Resources/functions")

import helper_functions as hf
import graphical_functions as gf

___
#### 0.2 CHARGEMENT DES DONNÉES

In [5]:
data = pd.read_csv("./Resources/datasets/assembled/full_training_data.csv").rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

___
#### 0.3 SUPPRESSION DES FEATURES NON PERTINENTES

Nous allons supprimer la colonne indiquant l'ID du client.

De plus, afin d'éviter de perpétuer des biais, nous allons supprimer la colonne ayant trait au sexe des clients.

In [6]:
cols_to_drop = ["SK_ID_CURR", "CODE_GENDER_M", "CODE_GENDER_F", "CODE_GENDER_XNA"]
data_model = data.drop(columns=cols_to_drop)

___
#### 0.4 SÉPARATION DES DONNÉES

Le jeu de données va être séparé entre données d'entraînement et données de test.

L'analyse exploratoire ayant fait ressortir un important déséquilibre des classes dans TARGET, nous devons veiller à maintenir ces proportions dans nos nouveaux jeux.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data_model.drop(columns=["TARGET"]), 
                                                    data_model["TARGET"], 
                                                    train_size=0.8, random_state=42, 
                                                    stratify=data_model["TARGET"])

___
### 1. SÉLECTION DES FEATURES

In [8]:
custom_score = make_scorer(hf.bank_score)

In [9]:
results = pd.DataFrame(columns=["Features", "Custom Score"])

___
#### 1.1 BASELINE : AUCUNE SÉLECTION


In [10]:
baseline_model = lgb.LGBMClassifier(n_estimators=10000, 
                                    objective = 'binary', 
                                    class_weight = 'balanced', 
                                    learning_rate = 0.05, 
                                    reg_alpha = 0.1, 
                                    reg_lambda = 0.1, 
                                    subsample = 0.8, 
                                    n_jobs = -1, 
                                    random_state = 42)

In [ ]:
baseline_model.fit(X_train, y_train, eval_metric=custom_score)

In [ ]:
y_pred = baseline_model.predict(X_test)

In [ ]:
results.loc[len(results)] = ["All features", 
                             round(hf.bank_score(y_test, y_pred), 3)]

___
#### 1.2 SUPPRESSION DES FEATURES COLINÉAIRES

Pour chaque paire de features colinéaires à plus de 90% (coefficient de spearman), l'une des 2 features est supprimée.

In [ ]:
# Threshold for removing correlated variables
threshold = 0.9

# Absolute value correlation matrix
corr_matrix = X_train.corr("spearman").abs()
corr_matrix.head()

In [ ]:
# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.head()

In [ ]:
# Select columns with correlations above threshold
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

print('There are %d columns to remove.' % (len(to_drop)))

In [ ]:
X_train_nc = X_train.drop(columns = to_drop)
X_test_nc = X_test.drop(columns = to_drop)

print('Training shape: ', X_train_nc.shape)
print('Testing shape: ', X_test_nc.shape)

In [ ]:
nc_model = lgb.LGBMClassifier(n_estimators=10000, 
                              objective = 'binary', 
                              class_weight = 'balanced', 
                              learning_rate = 0.05, 
                              reg_alpha = 0.1, 
                              reg_lambda = 0.1, 
                              subsample = 0.8, 
                              n_jobs = -1, 
                              random_state = 42)

In [ ]:
nc_model.fit(X_train_nc, 
          y_train, 
          eval_metric = custom_score)

# Make predictions
y_pred = nc_model.predict(X_test_nc)

row = ["Without collinear", 
       hf.bank_score(y_test, y_pred)]

results.loc[len(results)] = row

___
#### 1.3 SUPPRESSION DES FEATURES PRÉSENTANT PLUS DE 75% DE VALEURS MANQUANTES

In [ ]:
THRESHOLD = 0.75

In [ ]:
# Train missing values (in percent)
X_train_collinear_missing = (X_train_nc.isnull().sum() / len(X_train_nc)).sort_values(ascending = False)

In [ ]:
X_train_collinear_missing = X_train_collinear_missing.index[X_train_collinear_missing > THRESHOLD]

In [ ]:
X_train_without_collinear_missing = X_train_nc.drop(columns= X_train_collinear_missing)
X_test_without_collinear_missing = X_test_nc.drop(columns= X_train_collinear_missing)

In [ ]:
ncm_model = lgb.LGBMClassifier(n_estimators=10000, 
                              objective = 'binary', 
                              class_weight = 'balanced', 
                              learning_rate = 0.05, 
                              reg_alpha = 0.1, 
                              reg_lambda = 0.1, 
                              subsample = 0.8, 
                              n_jobs = -1, 
                              random_state = 42)

In [ ]:
ncm_model.fit(X_train_without_collinear_missing, 
          y_train, 
          eval_metric = custom_score)

# Make predictions
y_pred = ncm_model.predict(X_test_without_collinear_missing)

row = ["Without collinear and missing", 
       hf.bank_score(y_test, y_pred)]

results.loc[len(results)] = row

___
#### 1.4 SUPPRESSION DES FEATURES AYANT UNE IMPORTANCE NULLE POUR LE MODÈLE

In [ ]:
train = X_train_without_collinear_missing
test = X_test_without_collinear_missing

In [ ]:
# Initialize an empty array to hold feature importances
feature_importances = np.zeros(train.shape[1])

In [ ]:
ncmnof_model = lgb.LGBMClassifier(n_estimators=10000, 
                              objective = 'binary', 
                              class_weight = 'balanced', 
                              learning_rate = 0.05, 
                              reg_alpha = 0.1, 
                              reg_lambda = 0.1, 
                              subsample = 0.8, 
                              n_jobs = -1, 
                              random_state = 42)

In [ ]:
# Fit the model twice to avoid overfitting
for i in range(2):
    
    # Train using early stopping
    ncmnof_model.fit(train, 
              y_train, 
              eval_metric = custom_score)
    
    # Record the feature importances
    feature_importances += ncmnof_model.feature_importances_

In [ ]:
# Average feature importances
feature_importances = feature_importances / 2
feature_importances = pd.DataFrame({'feature': list(train.columns), 
                                    'importance': feature_importances}).sort_values('importance', ascending = False)

feature_importances.head()

In [ ]:
# Find the features with zero importance
zero_features = list(feature_importances[feature_importances['importance'] == 0.0]['feature'])
print('There are %d features with 0.0 importance' % len(zero_features))
feature_importances.tail(len(zero_features))

In [ ]:
norm_feature_importances = gf.plot_feature_importances2(feature_importances)

In [ ]:
train = train.drop(columns = zero_features)
test = test.drop(columns = zero_features)

print('Training shape: ', train.shape)
print('Testing shape: ', test.shape)

In [ ]:
second_round_zero_features, feature_importances = hf.identify_zero_importance_features(train, y_train)

In [ ]:
norm_feature_importances = gf.plot_feature_importances2(feature_importances)

In [ ]:
train = train.drop(columns=second_round_zero_features)
test = test.drop(columns=second_round_zero_features)

In [ ]:
print('Training shape: ', train.shape)
print('Testing shape: ', test.shape)

In [ ]:
third_round_zero_features, feature_importances = hf.identify_zero_importance_features(train, y_train)

In [ ]:
norm_feature_importances = gf.plot_feature_importances2(feature_importances)

In [ ]:
ncmnof_model.fit(train, 
          y_train, 
          eval_metric = custom_score)

# Make predictions
y_pred = ncmnof_model.predict(test)

row = ["Without all + 0 importance features", 
       hf.bank_score(y_test, y_pred)]

results.loc[len(results)] = row

___
#### 1.5 SUPPRESSION DES FEATURES AYANT UNE IMPORTANCE INFÉRIEURE À 95% POUR LE MODÈLE


In [ ]:
# Threshold for cumulative importance
THRESHOLD = 0.95

# Extract the features to keep
features_to_keep = list(norm_feature_importances[norm_feature_importances['cumulative_importance'] < THRESHOLD]['feature'])

# Create new datasets with smaller features
train_small = train[features_to_keep]
test_small = test[features_to_keep]

In [ ]:
ncmnofif_model = lgb.LGBMClassifier(n_estimators=10000, 
                              objective = 'binary', 
                              class_weight = 'balanced', 
                              learning_rate = 0.05, 
                              reg_alpha = 0.1, 
                              reg_lambda = 0.1, 
                              subsample = 0.8, 
                              n_jobs = -1, 
                              random_state = 42)

In [ ]:
ncmnofif_model.fit(train_small, 
          y_train, 
          eval_metric = custom_score)

In [ ]:
# Make predictions
y_pred = ncmnofif_model.predict(test_small)

row = ["Without 0 importance features and small", 
       hf.bank_score(y_test, y_pred)]

results.loc[len(results)] = row

___
#### 1.6 COMPARAISON DES PERFORMANCES


In [ ]:
results

Le meilleur score est obtenu en supprimant :
-   les features colinéaires
-   les features avec plus de 75% de valeurs manquantes
-   les features ayant 0 importance pour le modèle

In [ ]:
best_model = ncmnof_model

___
### 2.CONCLUSION

___

À ce stade, notre modèle possède les caractéristiques suivantes : 
- algorithme : Light Gradient Boosting Machine
- stratégie de rééquilibrage : class_weight = 'balanced'
- features : sélectionnée dans ce notebook

Notre modèle obtient un score de 0.109, soit 10.9% de mieux que la baseline consistant à prédire systématiquement que le client remboursera son crédit.

Afin de finaliser notre modèle et d'en améliorer les performances, la dernière étape consiste à optimiser les hyperparamètres.

___
### 3. EXPORT DES DONNÉES
___

In [ ]:
# Saving the best model and the dataset to save time
dump(best_model, "lgbm_best_features_model.joblib")
train.to_csv("./Resources/datasets/assembled/train.csv")
y_train.to_csv("./Resources/datasets/assembled/y_train.csv")
test.to_csv("./Resources/datasets/assembled/test.csv") 
y_test.to_csv("./Resources/datasets/assembled/y_test.csv")

# Saving columns for new data pipeline
dump(train.columns, "model_features.joblib"
